In [ ]:
# --- Config (edit these if needed) ---
weights_path = "weights/best.pt"           # put your weight file here after download
video_path   = "assets/Traffic Control CCTV.mp4"       # add a small sample or tell users to provide their own
conf_thresh  = 0.5

# Optional: gate training so "Run All" won't accidentally start it
RUN_TRAINING = False


In [ ]:
"""
Training
- I trained locally for 20 epochs to validate the pipeline.
- Final 100-epoch training was done by my supervisor on a GPU with ~3000 images.
- This cell is OFF by default to avoid accidental runs and dataset path issues.
"""
if RUN_TRAINING:
    from ultralytics import YOLO
    # use a relative/placeholder path; don't hardcode D:/...
    data_yaml = "data/data.yaml"  # <-- tell users how to prepare this if they have the dataset
    model = YOLO("yolov8n.pt")    # base model
    model.train(data=data_yaml, epochs=20)


In [19]:
import cv2
import cvzone
import math
from ultralytics import YOLO
from paddleocr import PaddleOCR



# Initialize OCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Video path
cap = cv2.VideoCapture(video_path)

# Load YOLO model
model = YOLO(weights_path)
classnames = ['license-plate', 'vehicle']

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 model
    frame = cv2.resize(frame, (1080, 720))
    results = model(frame, verbose=False)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])

            if conf > conf_thresh and classnames[cls] == 'license-plate':
                # Crop the license plate
                plate_crop = frame[y1:y2, x1:x2]

                # Only run OCR if crop is valid
                plate_text = "Reading..."
                if plate_crop.size > 0:
                    ocr_result = ocr.ocr(plate_crop, cls=True)
                    if ocr_result and isinstance(ocr_result[0], list):
                        plate_text = ' '.join([line[1][0] for line in ocr_result[0]])
                    else:
                        plate_text = "Unreadable"

                # Draw rectangle and text
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cvzone.putTextRect(frame, plate_text, [x1 + 5, y1 - 10], scale=1, thickness=2, colorR=(0, 255, 0))

    # Show the frame
    cv2.imshow("frame", frame)

    # Press 't' to stop
    if cv2.waitKey(1) & 0xFF == ord('t'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


[2025/09/05 15:59:15] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\USER/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\USER/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e

FileNotFoundError: [Errno 2] No such file or directory: 'weights\\best.pt'